# dataloader

https://github.com/pytorch/pytorch/blob/master/torch/csrc/api/include/torch/data/dataloader.h

In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <torch/torch.h>

## step1: create custom dataset

In [3]:
int sample_size = 5;
int number_features = 3;

torch::Tensor features = torch::arange(sample_size*number_features).reshape({sample_size,number_features}) * 1.0;
std::cout << features << std::endl;


  0   1   2
  3   4   5
  6   7   8
  9  10  11
 12  13  14
[ CPUFloatType{5,3} ]


In [4]:
torch::Tensor label = torch::randint(/*low=*/0, /*high=*/3, /*size=*/{sample_size,});
std::cout << label << std::endl;

 1
 2
 0
 2
 0
[ CPUFloatType{5} ]


In [5]:
class CustomDataset : torch::data::Dataset<CustomDataset>{
    private:
        torch::Tensor __features;
        torch::Tensor __labels;
    
    public:
    CustomDataset(torch::Tensor features, torch::Tensor labels):__features(features),__labels(labels){
        
    }
    
    torch::data::Example<> get(size_t index){
        torch::Tensor sample_features = __features[index];
        torch::Tensor sample_label = __labels[index];
        return {sample_features.clone(), sample_label.clone()};
    }
    
    torch::optional <size_t> size() const {
        return __features.sizes()[0];
    }
}

In [6]:
CustomDataset dataset(features, label);

## step2: create data loader

In [9]:
int batch_size = 2;
torch::data::DataLoaderOptions options = torch::data::DataLoaderOptions(batch_size).drop_last(false);

In [10]:
//auto data_loader = torch::data::make_data_loader<torch::data::samplers::SequentialSampler>(std::move(dataset), options);

input_line_16:2:21: error: no matching function for call to 'make_data_loader'
  ...torch::data::make_data_loader<torch::data::samplers::SequentialSampler>(...
     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../libtorch/include/torch/csrc/api/include/torch/data/dataloader.h:38:1: note: candidate template ignored: substitution failure [with Sampler =
      torch::data::samplers::SequentialSampler, Dataset =
      __cling_N54::CustomDataset]: 'is_stateful' is a private member of
      'torch::data::datasets::BatchDataset<__cling_N54::CustomDataset,
      std::vector<torch::data::Example<at::Tensor, at::Tensor>,
      std::allocator<torch::data::Example<at::Tensor, at::Tensor> > >,
      c10::ArrayRef<unsigned long> >'
make_data_loader(
^
../../libtorch/include/torch/csrc/api/include/torch/data/dataloader.h:53:46: note: candidate template ignored: substitution failure [with Dataset =
      torch::data::samplers::SequentialSampler]: no member named 'is_state

Interpreter Error: 